In [108]:
from pymongo import MongoClient
from flask import Flask, request, jsonify
import json
import requests
import pandas as pd
import csv

In [85]:
mongo = MongoClient(port=27017)
print(mongo.list_database_names())

['admin', 'classDB', 'config', 'epa', 'establishments', 'fruits_db', 'fruitsdb', 'local', 'met', 'petsitly_marketing', 'travel_db', 'uk_food', 'weather_database']


In [89]:
mongo = MongoClient('mongodb://localhost:27017')
db = mongo['player2_weather']
charleston_air = db['charleston_air']
endpoint = 'https://api.stormglass.io/v2/weather/point'
params = {
            'lat': 26.443398,
            'lng': -82.111511,
            'params': 'airTemperature',
            'start': '2023-09-23',
            'end': '2023-09-30'
            
        }
headers = {
        'Authorization': 'f83644f0-b99c-11ee-b7a8-0242ac130002-f8364590-b99c-11ee-b7a8-0242ac130002'
    }
response = requests.get(endpoint, params=params, headers=headers)
response.raise_for_status()
data = response.json()
sanibel_air.insert_one(data)
if sanibel_air.count_documents({}) > 0:
    print("Data inserted into sanibel_data collection.")
else:
    print("Failed to insert data.")


Data inserted into sanibel_data collection.


In [90]:
data = sanibel_air.find()
for document in data:
    pprint(document)

{'_id': ObjectId('65b9539ce5af0f131d426418'),
 'hours': [{'airTemperature': {'noaa': 28.42, 'sg': 28.42},
            'time': '2021-09-23T00:00:00+00:00'},
           {'airTemperature': {'noaa': 28.25, 'sg': 28.25},
            'time': '2021-09-23T01:00:00+00:00'},
           {'airTemperature': {'noaa': 28.08, 'sg': 28.08},
            'time': '2021-09-23T02:00:00+00:00'},
           {'airTemperature': {'noaa': 27.91, 'sg': 27.91},
            'time': '2021-09-23T03:00:00+00:00'},
           {'airTemperature': {'noaa': 27.56, 'sg': 27.56},
            'time': '2021-09-23T04:00:00+00:00'},
           {'airTemperature': {'noaa': 27.21, 'sg': 27.21},
            'time': '2021-09-23T05:00:00+00:00'},
           {'airTemperature': {'noaa': 26.86, 'sg': 26.86},
            'time': '2021-09-23T06:00:00+00:00'},
           {'airTemperature': {'noaa': 26.79, 'sg': 26.79},
            'time': '2021-09-23T07:00:00+00:00'},
           {'airTemperature': {'noaa': 26.72, 'sg': 26.72},
            't

In [99]:
data = list(sanibel_air.find())
pprint(data)
json_data = json.dumps(data, default=str)
with open('Resources/sanibel_air.json', 'w') as json_file:
    json_file.write(json_data)

[{'_id': ObjectId('65b9539ce5af0f131d426418'),
  'hours': [{'airTemperature': {'noaa': 28.42, 'sg': 28.42},
             'time': '2021-09-23T00:00:00+00:00'},
            {'airTemperature': {'noaa': 28.25, 'sg': 28.25},
             'time': '2021-09-23T01:00:00+00:00'},
            {'airTemperature': {'noaa': 28.08, 'sg': 28.08},
             'time': '2021-09-23T02:00:00+00:00'},
            {'airTemperature': {'noaa': 27.91, 'sg': 27.91},
             'time': '2021-09-23T03:00:00+00:00'},
            {'airTemperature': {'noaa': 27.56, 'sg': 27.56},
             'time': '2021-09-23T04:00:00+00:00'},
            {'airTemperature': {'noaa': 27.21, 'sg': 27.21},
             'time': '2021-09-23T05:00:00+00:00'},
            {'airTemperature': {'noaa': 26.86, 'sg': 26.86},
             'time': '2021-09-23T06:00:00+00:00'},
            {'airTemperature': {'noaa': 26.79, 'sg': 26.79},
             'time': '2021-09-23T07:00:00+00:00'},
            {'airTemperature': {'noaa': 26.72, 'sg': 26.7

In [105]:
mongo = MongoClient('mongodb://localhost:27017')
db = mongo['player2_weather']
charleston_air = db['charleston_air']
endpoint = 'https://api.stormglass.io/v2/weather/point'
params = {
            'lat': 32.7765,
            'lng': -79.93092,
            'params': 'airTemperature',
            'start': '2023-09-23',
            'end': '2023-09-30'
            
        }
headers = {
        'Authorization': 'f83644f0-b99c-11ee-b7a8-0242ac130002-f8364590-b99c-11ee-b7a8-0242ac130002'
    }
response = requests.get(endpoint, params=params, headers=headers)
response.raise_for_status()
data = response.json()
charleston_air.insert_one(data)
if charleston_air.count_documents({}) > 0:
    print("Data inserted into charleston collection.")
else:
    print("Failed to insert data.")

Data inserted into charleston collection.


In [106]:
data = charleston_air.find()
for document in data:
    pprint(document)

{'_id': ObjectId('65b95a2ee5af0f131d42641e'),
 'hours': [{'airTemperature': {'noaa': 24.52, 'sg': 24.52},
            'time': '2021-09-23T00:00:00+00:00'},
           {'airTemperature': {'noaa': 24.35, 'sg': 24.35},
            'time': '2021-09-23T01:00:00+00:00'},
           {'airTemperature': {'noaa': 24.18, 'sg': 24.18},
            'time': '2021-09-23T02:00:00+00:00'},
           {'airTemperature': {'noaa': 24.01, 'sg': 24.01},
            'time': '2021-09-23T03:00:00+00:00'},
           {'airTemperature': {'noaa': 23.49, 'sg': 23.49},
            'time': '2021-09-23T04:00:00+00:00'},
           {'airTemperature': {'noaa': 22.98, 'sg': 22.98},
            'time': '2021-09-23T05:00:00+00:00'},
           {'airTemperature': {'noaa': 22.46, 'sg': 22.46},
            'time': '2021-09-23T06:00:00+00:00'},
           {'airTemperature': {'noaa': 20.93, 'sg': 20.93},
            'time': '2021-09-23T07:00:00+00:00'},
           {'airTemperature': {'noaa': 19.39, 'sg': 19.39},
            't

In [107]:
data = list(charleston_air.find())
pprint(data)
json_data = json.dumps(data, default=str)
with open('Resources/charleston_air.json', 'w') as json_file:
    json_file.write(json_data)

[{'_id': ObjectId('65b95a2ee5af0f131d42641e'),
  'hours': [{'airTemperature': {'noaa': 24.52, 'sg': 24.52},
             'time': '2021-09-23T00:00:00+00:00'},
            {'airTemperature': {'noaa': 24.35, 'sg': 24.35},
             'time': '2021-09-23T01:00:00+00:00'},
            {'airTemperature': {'noaa': 24.18, 'sg': 24.18},
             'time': '2021-09-23T02:00:00+00:00'},
            {'airTemperature': {'noaa': 24.01, 'sg': 24.01},
             'time': '2021-09-23T03:00:00+00:00'},
            {'airTemperature': {'noaa': 23.49, 'sg': 23.49},
             'time': '2021-09-23T04:00:00+00:00'},
            {'airTemperature': {'noaa': 22.98, 'sg': 22.98},
             'time': '2021-09-23T05:00:00+00:00'},
            {'airTemperature': {'noaa': 22.46, 'sg': 22.46},
             'time': '2021-09-23T06:00:00+00:00'},
            {'airTemperature': {'noaa': 20.93, 'sg': 20.93},
             'time': '2021-09-23T07:00:00+00:00'},
            {'airTemperature': {'noaa': 19.39, 'sg': 19.3

In [114]:
data = charleston_air.find()
for document in data:
    pprint(document) 

{'_id': ObjectId('65b95a2ee5af0f131d42641e'),
 'hours': [{'airTemperature': {'noaa': 24.52, 'sg': 24.52},
            'time': '2021-09-23T00:00:00+00:00'},
           {'airTemperature': {'noaa': 24.35, 'sg': 24.35},
            'time': '2021-09-23T01:00:00+00:00'},
           {'airTemperature': {'noaa': 24.18, 'sg': 24.18},
            'time': '2021-09-23T02:00:00+00:00'},
           {'airTemperature': {'noaa': 24.01, 'sg': 24.01},
            'time': '2021-09-23T03:00:00+00:00'},
           {'airTemperature': {'noaa': 23.49, 'sg': 23.49},
            'time': '2021-09-23T04:00:00+00:00'},
           {'airTemperature': {'noaa': 22.98, 'sg': 22.98},
            'time': '2021-09-23T05:00:00+00:00'},
           {'airTemperature': {'noaa': 22.46, 'sg': 22.46},
            'time': '2021-09-23T06:00:00+00:00'},
           {'airTemperature': {'noaa': 20.93, 'sg': 20.93},
            'time': '2021-09-23T07:00:00+00:00'},
           {'airTemperature': {'noaa': 19.39, 'sg': 19.39},
            't

In [127]:
data = charleston_air.find_one({}, {"_id": 0, "hours": 1})['hours']
csv_file = 'Resources/chairtemps.csv'
fields = ['time', 'airTemperature.noaa', 'airTemperature.sg']
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(fields)
    
    for entry in data:
        row_data = [
            entry['time'],
            entry['airTemperature']['noaa'],
            entry['airTemperature']['sg']
        ]
        writer.writerow(row_data)

In [128]:
data = sanibel_air.find_one({}, {"_id": 0, "hours": 1})['hours']
csv_file = 'Resources/sanibelairtemps.csv'
fields = ['time', 'airTemperature.noaa', 'airTemperature.sg']
with open(csv_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(fields)
    
    for entry in data:
        row_data = [
            entry['time'],
            entry['airTemperature']['noaa'],
            entry['airTemperature']['sg']
        ]
        writer.writerow(row_data)